<a href="https://colab.research.google.com/github/GiammarcoBozzelli/DSML/blob/wip/20250519_Camembert_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
#imoprt packages
%%capture
!pip install transformers[torch] accelerate -U
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import svm
import time
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from torch.utils.data import DataLoader

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from transformers import CamembertTokenizer,CamembertForSequenceClassification, CamembertModel, pipeline
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the dataset
url = "https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/training_data.csv"
augmented_df = pd.read_csv(url)
# data = pd.read_csv(url)

In [14]:
# # Install necessary libraries
# !pip install transformers torch


from transformers import CamembertTokenizer, CamembertModel
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Using device:", device)


# Load model and tokenizer
model_name = "camembert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# # Load tokenizer and model
# from transformers import AutoModel, AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("dbmdz/electra-base-french-europeana-cased-discriminator")
# model = AutoModel.from_pretrained("dbmdz/electra-base-french-europeana-cased-discriminator").to(device)

Using device: cuda


In [15]:
# # Function to generate sentences
# def generate_sentence(seed_text):
#     inputs = tokenizer(seed_text, return_tensors="pt")
#     outputs = model.generate(inputs["input_ids"], max_length=200, num_return_sequences=1)
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)

# # Generating new sentences for each difficulty level
# new_sentences = []
# for level in data['difficulty']:
#     # Randomly select a sentence from the same difficulty level
#     seed_text = data[data['difficulty'] == level].sample(1)['sentence'].iloc[0]
#     generated_sentence = generate_sentence(seed_text)
#     new_sentences.append({'sentence': generated_sentence, 'difficulty': level})

# # Append new sentences to the DataFrame
# new_df = pd.DataFrame(new_sentences)
# augmented_df = pd.concat([data, new_df], ignore_index=True)

# # Saving the augmented DataFrame
# # augmented_df.to_csv('augmented_dataset.csv', index=False)

In [16]:
# augmented_df = pd.read_csv('https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/augmented_dataset.csv')

In [17]:
def remove_duplicate_words(sentence):
  '''
  Function necessary to remove duplicated words in sentences generated by gpt-2 that made no sense.
  '''
  words = sentence.split()
  seen = set()
  unique_words = []
  for word in words:
      if word not in seen:
          unique_words.append(word)
          seen.add(word)
  return ' '.join(unique_words)

# Apply the function to the 'sentence' column
augmented_df.loc[4800:, 'sentence'] = augmented_df.loc[4800:, 'sentence'].apply(remove_duplicate_words)

In [18]:
# Function to fix the 'id' column
def fix_id_column(df):
    # Create a sequence of row numbers starting from 0
    correct_ids = pd.Series(range(len(df)))
    # Replace 'NaN' values and incorrect ids
    df['id'] = correct_ids
    return df

# Fix the 'id' column
augmented_df = fix_id_column(augmented_df)

In [19]:
import nltk
from nltk.corpus import wordnet
from random import randint
import random

nltk.download('wordnet')
nltk.download('omw-1.4')

def synonym_replacement(sentence, n):
    words = sentence.split()
    new_words = words.copy()
    random_word_list = list(set([word for word in words if wordnet.synsets(word)]))
    num_replaced = 0

    for random_word in random_word_list:
        synonyms = set()
        for syn in wordnet.synsets(random_word):
            for lemma in syn.lemmas():
                synonym = lemma.name().replace("_", " ").replace("-", " ").lower()
                synonyms.add(synonym)
        if len(synonyms) > 1:
            synonyms.discard(random_word)
            synonym = list(synonyms)[randint(0, len(synonyms) - 1)]
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:  # only replace up to n words
            break

    return ' '.join(new_words)

def shuffle_sentence(sentence):
    words = sentence.split()
    random.shuffle(words)
    return ' '.join(words)


augmented_data = []

# Augment data
for _, row in augmented_df.iterrows():
    original_sentence = row['sentence']
    difficulty = row['difficulty']

    # Generate augmented sentences
    augmented_sentence_synonym = synonym_replacement(original_sentence, 2)
    augmented_sentence_shuffled = shuffle_sentence(original_sentence) #idk


    # Append original and augmented sentences to the new list
    augmented_data.append({'sentence': original_sentence, 'difficulty': difficulty})
    augmented_data.append({'sentence': augmented_sentence_synonym, 'difficulty': difficulty})
    augmented_data.append({'sentence': augmented_sentence_shuffled, 'difficulty': difficulty})


# Create a new DataFrame from the augmented data
data = pd.DataFrame(augmented_data)

In [20]:
# Preprocess the data: encode labels
label_encoder = LabelEncoder()
data['Label'] = label_encoder.fit_transform(data['difficulty'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['sentence'], data['Label'], test_size=0.05, random_state=42)

In [21]:
# from transformers import CamembertTokenizer, CamembertModel
# import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print("Using device:", device)

# Load tokenizer and model
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertModel.from_pretrained("camembert-base").to(device)


In [22]:
def get_embeddings(sentences):
    # Ensure sentences are in a list and filter out any non-string entries
    sentences = [s for s in sentences if isinstance(s, str)]

    # Check if the batch is empty after filtering
    if not sentences:
        return np.array([])  # Return an empty array if no valid sentences are present

    # Tokenize and encode the batch of sentences
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

    # Get embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the embeddings from the last hidden state
    embeddings = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
    return embeddings

# Batch processing of embeddings
batch_size = 32  # Adjust batch size according to your GPU memory
X_train_embeddings = np.vstack([get_embeddings(X_train[i:i + batch_size]) for i in range(0, len(X_train), batch_size) if len(X_train[i:i + batch_size]) > 0])
X_test_embeddings = np.vstack([get_embeddings(X_test[i:i + batch_size]) for i in range(0, len(X_test), batch_size) if len(X_test[i:i + batch_size]) > 0])


In [ ]:
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train_embeddings, y_train)

# Predict on the test set
y_pred = clf.predict(X_test_embeddings)

''' dbmdz/electra-base-french-europeana-cased-discriminator Linear SVC
Accuracy: 0.55
Precision: 0.56
Recall: 0.55
F1 Score: 0.55
'''


''' CamemBert LinearSVC
Accuracy: 0.66
Precision: 0.66
Recall: 0.66
F1 Score: 0.66
'''

''' CamemBert LinearSVR()
Accuracy: 0.478
'''

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
clf = OneVsRestClassifier(svm.LinearSVC(), n_jobs=-1).fit(X_train_embeddings, y_train)
# Predict on the test set
y_pred = clf.predict(X_test_embeddings)

# ''' dbmdz/electra-base-french-europeana-cased-discriminator OneVsRestClassifier(svm.LinearSVC())
# Accuracy: 0.52
# Precision: 0.58
# Recall: 0.52
# F1 Score: 0.51
# '''

# ''' CamemBert OneVsRestlinearSVC
# Accuracy: 0.67
# Precision: 0.67
# Recall: 0.67
# F1 Score: 0.67
# '''

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, classification_report

accuracy = accuracy_score(y_test, y_pred.round())
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print the results
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

# Distillbert

In [ ]:
 %%capture
 !pip install transformers[torch] accelerate -U

In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from torch.utils.data import DataLoader

# Load the dataset
# url = "https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/training_data.csv"
# data = pd.read_csv(url)

# Map labels to integers
label_mapping = {'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}
data['difficulty'] = data['difficulty'].map(label_mapping)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(data['sentence'], data['difficulty'], test_size=0.2, random_state=42)

# Tokenization
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True)

# class ClassificationDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels

#     def __getitem__(self, idx):
#         item = {key: val[idx] for key, val in self.encodings.items()}
#         if self.labels is not None:
#             item['labels'] = torch.tensor(self.labels[idx])
#         return item

#     def __len__(self):
#         return len(self.labels)

class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Set the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Prepare datasets
train_dataset = ClassificationDataset(train_encodings, y_train.tolist())
test_dataset = ClassificationDataset(test_encodings, y_test.tolist())

# Load pre-trained model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=6).to(device)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.05,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    learning_rate = 0.001,
    fp16=True
)

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train the model
trainer.train()

# Evaluate the model
predictions, label_ids, metrics = trainer.predict(test_dataset)
preds = predictions.argmax(-1)

# Calculate metrics
accuracy = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds, average='weighted')
recall = recall_score(y_test, preds, average='weighted')
f1 = f1_score(y_test, preds, average='weighted')

# Print the results
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

# Alternatively, print a detailed classification report
print(classification_report(y_test, preds, target_names=['A1', 'A2', 'B1', 'B2', 'C1', 'C2']))

In [ ]:
# Load the unlabeled data
unlabelled_url = "https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/unlabelled_test_data.csv"
unlabelled_data = pd.read_csv(unlabelled_url)

# Tokenization
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

def preprocess_text(text_list):
    return tokenizer(text_list, truncation=True, padding=True)

unlabelled_encodings = preprocess_text(unlabelled_data['sentence'].tolist())

# Set the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Prepare dataset
unlabelled_dataset = ClassificationDataset(unlabelled_encodings)

# Predict function
predictions = trainer.predict(unlabelled_dataset).predictions
preds = predictions.argmax(-1)
# Map numerical predictions back to difficulty levels
label_mapping = {0: 'A1', 1: 'A2', 2: 'B1', 3: 'B2', 4: 'C1', 5: 'C2'}
preds_labels = [label_mapping[pred.argmax()] for pred in predictions]

# Add predictions to the dataframe
unlabelled_data['difficulty'] = preds_labels

# unlabelled_data.drop(columns=['sentence'],inplace=True)
# Save the results to a new CSV file if needed
unlabelled_data.to_csv('predicted_unlabelled_data.csv', index=False)

In [125]:
# Map numerical predictions back to difficulty levels
label_mapping = {0: 'A1', 1: 'A2', 2: 'B1', 3: 'B2', 4: 'C1', 5: 'C2'}
preds_labels = [label_mapping[pred.argmax()] for pred in predictions]

# Add predictions to the dataframe
unlabelled_data['difficulty'] = preds_labels

# unlabelled_data.drop(columns=['sentence'],inplace=True)
# Save the results to a new CSV file if needed
unlabelled_data.to_csv('predicted_unlabelled_data.csv', index=False)

In [2]:
unlabelled_url = "https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/unlabelled_test_data.csv"
unlabelled_data = pd.read_csv(unlabelled_url)

In [ ]:
trainer.predict(test_dataset)
preds = predictions.argmax(-1)

In [50]:
unlabelled_embeddings = np.vstack([get_embeddings(unlabelled_data['sentence'][i:i + batch_size]) for i in range(0, len(unlabelled_data), batch_size) if len(unlabelled_data['sentence'][i:i + batch_size]) > 0])

In [ ]:
predicted_labels = clf.predict(unlabelled_embeddings)

# Decode the predicted labels back to original labels
decoded_labels = label_encoder.inverse_transform(predicted_labels)

In [3]:
submission = pd.DataFrame({'id': unlabelled_data['id'], 'difficulty': decoded_labels})
current_time = time.time()

submission.to_csv(f'{current_time}.csv', index=False)

NameError: name 'unlabelled_data' is not defined

## Embeddings

In [3]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [2]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import pandas as pd

# Load the dataset
url = "https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/training_data.csv"
data = pd.read_csv(url)

# Map labels to integers
label_mapping = {'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}
data['difficulty'] = data['difficulty'].map(label_mapping)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(data['sentence'], data['difficulty'], test_size=0.2, random_state=42)

# Tokenization
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, return_tensors='pt')

class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Prepare datasets
train_dataset = ClassificationDataset(train_encodings, y_train.tolist())
test_dataset = ClassificationDataset(test_encodings, y_test.tolist())

# Load pre-trained model for classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=6)

# Set the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train the model
trainer.train()

# Evaluate the model
predictions, label_ids, metrics = trainer.predict(test_dataset)
preds = predictions.argmax(-1)

# Calculate metrics
accuracy = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds, average='weighted')
recall = recall_score(y_test, preds, average='weighted')
f1 = f1_score(y_test, preds, average='weighted')

# Print the results
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

# Alternatively, print a detailed classification report
print(classification_report(y_test, preds, target_names=['A1', 'A2', 'B1', 'B2', 'C1', 'C2']))


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.240500,1.268033
2,1.092500,1.103746
3,0.805100,1.161130


Accuracy: 0.57
Precision: 0.57
Recall: 0.57
F1 Score: 0.57
              precision    recall  f1-score   support

          A1       0.68      0.78      0.73       166
          A2       0.53      0.44      0.48       158
          B1       0.54      0.63      0.58       166
          B2       0.51      0.52      0.51       153
          C1       0.52      0.51      0.51       152
          C2       0.64      0.53      0.58       165

    accuracy                           0.57       960
   macro avg       0.57      0.57      0.57       960
weighted avg       0.57      0.57      0.57       960



In [7]:
# Load the unlabeled data
unlabelled_url = "https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/unlabelled_test_data.csv"
unlabelled_data = pd.read_csv(unlabelled_url)

# Tokenization
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

def preprocess_text(text_list):
    return tokenizer(text_list, truncation=True, padding=True)

unlabelled_encodings = preprocess_text(unlabelled_data['sentence'].tolist())

# Set the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Prepare dataset
unlabelled_dataset = ClassificationDataset(unlabelled_encodings)

# Predict function
predictions = trainer.predict(unlabelled_dataset).predictions
preds = predictions.argmax(-1)
# Map numerical predictions back to difficulty levels
label_mapping = {0: 'A1', 1: 'A2', 2: 'B1', 3: 'B2', 4: 'C1', 5: 'C2'}
preds_labels = [label_mapping[pred.argmax()] for pred in predictions]

# Add predictions to the dataframe
unlabelled_data['difficulty'] = preds_labels

# unlabelled_data.drop(columns=['sentence'],inplace=True)
# Save the results to a new CSV file if needed
current_time = time.time()

unlabelled_data.to_csv(f'{current_time}.csv', index=False)

# PCA

In [13]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn import svm
# Load the dataset
url = "https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/training_data.csv"
data = pd.read_csv(url)

# Map labels to integers
label_mapping = {'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}
data['difficulty'] = data['difficulty'].map(label_mapping)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(data['sentence'], data['difficulty'], test_size=0.2, random_state=42)

# Tokenization
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, return_tensors='pt')

class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Prepare datasets
train_dataset = ClassificationDataset(train_encodings, y_train.tolist())
test_dataset = ClassificationDataset(test_encodings, y_test.tolist())

# Load pre-trained model for feature extraction
model = DistilBertModel.from_pretrained('distilbert-base-multilingual-cased')
model.eval()  # Set the model to evaluation mode

# Set the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

def extract_features(dataset):
    features = []
    labels = []
    with torch.no_grad():
        for i in range(len(dataset)):
            input_ids = dataset[i]['input_ids'].unsqueeze(0).to(device)
            attention_mask = dataset[i]['attention_mask'].unsqueeze(0).to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            hidden_states = outputs.last_hidden_state.mean(dim=1)  # Average pooling
            features.append(hidden_states.cpu().numpy())
            if 'labels' in dataset[i]:
                labels.append(dataset[i]['labels'].item())
    return np.vstack(features), labels

# Extract features
import numpy as np
train_features, train_labels = extract_features(train_dataset)
test_features, test_labels = extract_features(test_dataset)

# Apply PCA
pca = PCA(n_components=50)
train_features_pca = pca.fit_transform(train_features)
test_features_pca = pca.transform(test_features)

# Train a classifier (Logistic Regression)
classifier = svm.SVC(max_iter=1000)
classifier.fit(train_features_pca, train_labels)

# Make predictions
preds = classifier.predict(test_features_pca)

# Calculate metrics
accuracy = accuracy_score(test_labels, preds)
precision = precision_score(test_labels, preds, average='weighted')
recall = recall_score(test_labels, preds, average='weighted')
f1 = f1_score(test_labels, preds, average='weighted')

# Print the results
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

# Alternatively, print a detailed classification report
print(classification_report(test_labels, preds, target_names=['A1', 'A2', 'B1', 'B2', 'C1', 'C2']))


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy: 0.50
Precision: 0.50
Recall: 0.50
F1 Score: 0.50
              precision    recall  f1-score   support

          A1       0.66      0.69      0.68       166
          A2       0.43      0.49      0.46       158
          B1       0.44      0.39      0.41       166
          B2       0.43      0.41      0.42       153
          C1       0.46      0.43      0.45       152
          C2       0.57      0.59      0.58       165

    accuracy                           0.50       960
   macro avg       0.50      0.50      0.50       960
weighted avg       0.50      0.50      0.50       960



In [56]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
import pandas as pd
import numpy as np

# # Load the dataset
# url = "https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/training_data.csv"
# data = pd.read_csv(url)

# Map labels to integers
label_mapping = {'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}
data['difficulty'] = data['difficulty'].map(label_mapping)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(data['sentence'], data['difficulty'], test_size=0.2, random_state=42)

# Tokenization
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, return_tensors='pt')

class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Prepare datasets
train_dataset = ClassificationDataset(train_encodings, y_train.tolist())
test_dataset = ClassificationDataset(test_encodings, y_test.tolist())

# Load pre-trained model for classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=6)

# Set the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Subclass Trainer to override compute_loss
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels").to(device)
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits.view(-1, model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# Define training arguments with hyperparameter tuning
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    load_best_model_at_end=True,
)

# Define custom trainer
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()

# Evaluate the model
predictions, label_ids, metrics = trainer.predict(test_dataset)
preds = predictions.argmax(-1)

# Calculate metrics
accuracy = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds, average='weighted')
recall = recall_score(y_test, preds, average='weighted')
f1 = f1_score(y_test, preds, average='weighted')

# Print the results
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

# Alternatively, print a detailed classification report
print(classification_report(y_test, preds, target_names=['A1', 'A2', 'B1', 'B2', 'C1', 'C2']))
# FILE /content/1716132247.6841972.csv

KeyboardInterrupt: 

In [26]:
# Load the unlabeled data
unlabelled_url = "https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/unlabelled_test_data.csv"
unlabelled_data = pd.read_csv(unlabelled_url)

# Tokenization
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

def preprocess_text(text_list):
    return tokenizer(text_list, truncation=True, padding=True)

unlabelled_encodings = preprocess_text(unlabelled_data['sentence'].tolist())

# Set the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Prepare dataset
unlabelled_dataset = ClassificationDataset(unlabelled_encodings)

# Predict function
predictions = trainer.predict(unlabelled_dataset).predictions
preds = predictions.argmax(-1)
# Map numerical predictions back to difficulty levels
label_mapping = {0: 'A1', 1: 'A2', 2: 'B1', 3: 'B2', 4: 'C1', 5: 'C2'}
preds_labels = [label_mapping[pred.argmax()] for pred in predictions]

# Add predictions to the dataframe
unlabelled_data['difficulty'] = preds_labels
unlabelled_data.drop(columns='sentence',inplace=True)
# Save the results to a new CSV file if needed
current_time = time.time()

unlabelled_data.to_csv(f'{current_time}.csv', index=False)

# CamemBERT

In [23]:
# Map labels to integers
label_mapping = {'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}
data['difficulty'] = data['difficulty'].map(label_mapping)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(data['sentence'], data['difficulty'], test_size=0.2, random_state=42)

# Tokenization
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True)

class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Set the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Prepare datasets
train_dataset = ClassificationDataset(train_encodings, y_train.tolist())
test_dataset = ClassificationDataset(test_encodings, y_test.tolist())

# Load pre-trained model
model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=6).to(device)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.05,
    logging_dir='./logs',
    logging_steps=10,
    eval_steps=120,
    evaluation_strategy="epoch",
    learning_rate = 0.00015,
    fp16=True
)

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train the model
trainer.train()

# Evaluate the model
predictions, label_ids, metrics = trainer.predict(test_dataset)
preds = predictions.argmax(-1)

# Calculate metrics
accuracy = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds, average='weighted')
recall = recall_score(y_test, preds, average='weighted')
f1 = f1_score(y_test, preds, average='weighted')

# Print the results
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

# Alternatively, print a detailed classification report
print(classification_report(y_test, preds, target_names=['A1', 'A2', 'B1', 'B2', 'C1', 'C2']))

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.038300,1.003860
2,0.802100,0.846212
3,0.440900,0.482669
4,0.129700,0.336412
5,0.061100,0.295727


Accuracy: 0.93
Precision: 0.93
Recall: 0.93
F1 Score: 0.93
              precision    recall  f1-score   support

          A1       0.91      0.92      0.92       512
          A2       0.90      0.88      0.89       472
          B1       0.90      0.94      0.92       458
          B2       0.96      0.96      0.96       476
          C1       0.93      0.94      0.94       451
          C2       0.97      0.93      0.95       511

    accuracy                           0.93      2880
   macro avg       0.93      0.93      0.93      2880
weighted avg       0.93      0.93      0.93      2880



In [39]:
model.cpu()
model.save_pretrained('./camembert_final_model')
tokenizer.save_pretrained('./camembert_final_model')

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [33]:
# Load the unlabeled data
unlabelled_url = "https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/unlabelled_test_data.csv"
unlabelled_data = pd.read_csv(unlabelled_url)

def preprocess_text(text_list):
    return tokenizer(text_list, truncation=True, padding=True)

unlabelled_encodings = preprocess_text(unlabelled_data['sentence'].tolist())

# Set the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Prepare dataset
unlabelled_dataset = ClassificationDataset(unlabelled_encodings)

# Predict function
predictions = trainer.predict(unlabelled_dataset).predictions
preds = predictions.argmax(-1)
# Map numerical predictions back to difficulty levels
label_mapping = {0: 'A1', 1: 'A2', 2: 'B1', 3: 'B2', 4: 'C1', 5: 'C2'}
preds_labels = [label_mapping[pred.argmax()] for pred in predictions]

# Add predictions to the dataframe
unlabelled_data['difficulty'] = preds_labels
unlabelled_data.drop(columns='sentence',inplace=True)
# Save the results to a new CSV file if needed
current_time = time.time()

unlabelled_data.to_csv(f'{current_time}.csv', index=False)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import KFold
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# from transformers import CamembertTokenizer, CamembertForSequenceClassification, TrainingArguments, Trainer
# import torch

# # Map labels to integers
# label_mapping = {'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}
# data['difficulty'] = data['difficulty'].map(label_mapping)

# # Initialize K-Fold
# kf = KFold(n_splits=5, shuffle=True, random_state=42)

# # Tokenization
# tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

# # Set the device
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# # Prepare for accumulating results
# accuracy_list = []
# precision_list = []
# recall_list = []
# f1_list = []

# class ClassificationDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings, labels=None):
#         self.encodings = encodings
#         self.labels = labels

#     def __getitem__(self, idx):
#         item = {key: val[idx] for key, val in self.encodings.items()}
#         if self.labels is not None:
#             item['labels'] = torch.tensor(self.labels[idx])
#         return item

#     def __len__(self):
#         return len(self.encodings['input_ids'])

# def compute_metrics(pred):
#     labels = pred.label_ids
#     preds = pred.predictions.argmax(-1)
#     accuracy = accuracy_score(labels, preds)
#     precision = precision_score(labels, preds, average='weighted')
#     recall = recall_score(labels, preds, average='weighted')
#     f1 = f1_score(labels, preds, average='weighted')
#     return {
#         'accuracy': accuracy,
#         'precision': precision,
#         'recall': recall,
#         'f1': f1,
#     }

# # Perform K-Fold Cross-Validation
# for fold, (train_idx, val_idx) in enumerate(kf.split(data)):
#     print(f"Training fold {fold+1}")
#     train_data = data.iloc[train_idx]
#     val_data = data.iloc[val_idx]

#     # Tokenize the data
#     train_encodings = tokenizer(train_data['sentence'].tolist(), truncation=True, padding=True)
#     val_encodings = tokenizer(val_data['sentence'].tolist(), truncation=True, padding=True)

#     # Create the datasets
#     train_dataset = ClassificationDataset(train_encodings, train_data['difficulty'].tolist())
#     val_dataset = ClassificationDataset(val_encodings, val_data['difficulty'].tolist())

#     # Load pre-trained model
#     model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=6).to(device)

#     # Define training arguments
#     training_args = TrainingArguments(
#         output_dir=f'./results_fold_{fold}',
#         learning_rate=0.0025,
#         num_train_epochs=1,
#         per_device_train_batch_size=16,
#         warmup_steps=1000,
#         weight_decay=0.01,
#         logging_dir=f'./logs_fold_{fold}',
#         logging_steps=10,
#         evaluation_strategy="steps",
#         eval_steps=100,
#         save_strategy="steps",
#         save_steps=500,
#         fp16=True,
#     )

#     # Define trainer
#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=val_dataset,
#         compute_metrics=compute_metrics
#     )

#     # Train the model
#     trainer.train()

#     # Evaluate the model
#     eval_result = trainer.evaluate()

#     # Store results
#     accuracy_list.append(eval_result['eval_accuracy'])
#     precision_list.append(eval_result['eval_precision'])
#     recall_list.append(eval_result['eval_recall'])
#     f1_list.append(eval_result['eval_f1'])

# # Output the results
# print("Average Accuracy:", np.mean(accuracy_list))
# print("Average Precision:", np.mean(precision_list))
# print("Average Recall:", np.mean(recall_list))
# print("Average F1 Score:", np.mean(f1_list))


In [ ]:
# Load the unlabeled data
unlabelled_url = "https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/unlabelled_test_data.csv"
unlabelled_data = pd.read_csv(unlabelled_url)

# Tokenization
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

def preprocess_text(text_list):
    return tokenizer(text_list, truncation=True, padding=True)

unlabelled_encodings = preprocess_text(unlabelled_data['sentence'].tolist())

# Set the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Prepare dataset
unlabelled_dataset = ClassificationDataset(unlabelled_encodings)

# Predict function
predictions = trainer.predict(unlabelled_dataset).predictions
preds = predictions.argmax(-1)
# Map numerical predictions back to difficulty levels
label_mapping = {0: 'A1', 1: 'A2', 2: 'B1', 3: 'B2', 4: 'C1', 5: 'C2'}
preds_labels = [label_mapping[pred.argmax()] for pred in predictions]

# Add predictions to the dataframe
unlabelled_data['difficulty'] = preds_labels
unlabelled_data.drop(columns='sentence',inplace=True)
# Save the results to a new CSV file if needed
current_time = time.time()

unlabelled_data.to_csv(f'{current_time}.csv', index=False)